In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from datetime import date, timedelta
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from collections import Counter
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from datetime import datetime

import time
import logging
import pandas as pd

In [33]:
def get_weather_data_for_dates7(start_year, start_month, start_day, end_year, end_month, end_day, browser):

    weather_data = {}
    current_date = datetime(start_year, start_month, start_day)
    end_date = datetime(end_year, end_month, end_day)
    problematic_urls = []
    while current_date <= end_date:
        try:
            year, month, day = current_date.year, current_date.month, current_date.day
            if not browser.current_window_handle: # Check if browser is still open
                browser.quit()
                time.sleep(15)
                browser = webdriver.Firefox()
                browser.maximize_window()
            date_data = get_weather_data4(year, month, day, browser)
            weather_data[current_date.strftime("%Y-%m-%d")] = date_data
        except Exception as e:
            max_retries = 1 # usually on - 5 tries 
            retries = 0
            while retries < max_retries:
                try:
                    browser.quit()
                    time.sleep(15)
                    browser = webdriver.Firefox()
                    browser.maximize_window()
                    date_data = get_weather_data4(year, month, day, browser)
                    weather_data[current_date.strftime("%Y-%m-%d")] = date_data
                    break
                except Exception as e:
                    retries += 1
                    time.sleep(30)
                    logging.error("An error occurred while getting data for date %s: %s", current_date, e)
                except SessionNotFoundException:
                    browser.quit()
                    time.sleep(15)
                    browser = webdriver.Firefox()
                    browser.maximize_window()
                    date_data = get_weather_data4(year, month, day, browser)
                    weather_data[current_date.strftime("%Y-%m-%d")] = date_data
            if retries == max_retries:
                current_url = browser.current_url
                browser.quit()
                logging.error("Maximum number of retries reached for date %s", current_date)
                problematic_urls.append((current_date.strftime("%Y-%m-%d"), current_url))
        current_date += timedelta(days=1)
    browser.quit()
    return weather_data, problematic_urls

In [34]:
def get_weather_data4( year, month, day, browser):
    try:
        # Open browser and navigate to the website
        #browser = webdriver.Chrome()
        wait = WebDriverWait(browser, 20)
        browser.get('https://www.wunderground.com/history') #https://www.wunderground.com/history/daily/il/haifa/LLHA
        wait.until(EC.presence_of_element_located((By.ID, 'yearSelection')))
        wait.until(EC.presence_of_element_located((By.ID, 'historySearch')))
        
        # Select year, month, and day from the drop-down menus
        actions = ActionChains(browser)
        year_select = Select(browser.find_element(By.ID,'yearSelection'))
        year_select.select_by_value(str((2023-year))+':'+' '+str(year))
        month_select = Select(browser.find_element(By.ID,'monthSelection'))
        month_select.select_by_value(str(month))
        day_select = Select(browser.find_element(By.ID,'daySelection'))
        day_select.select_by_value(str((day-1))+':'+' '+str(day))
     
        search_field = wait.until(EC.visibility_of_element_located((By.ID, 'historySearch')))
        search_field.click()
        search_field.send_keys("New York, NY", Keys.ENTER)
        #search_field.send_keys(Keys.TAB * 2 )#+ Keys.RETURN * 2)
     

        try:
            wait.until(EC.element_to_be_clickable((By.ID, 'dateSubmit')))
            button_view = browser.find_element(By.ID, 'dateSubmit')
            #button_view.send_keys("",Keys.ENTER)
            actions.move_to_element(button_view).double_click(button_view)
        except TimeoutError as te:
            print(f"The following element caused a timeout error: {te}")
       
    # Click the "View" button to load the data for the selected date
        #button_view = wait.until(EC.element_to_be_clickable((By.ID, 'dateSubmit')))
        #actions.move_to_element(button_view).click().perform()
        #button_view.click()
        #browser.find_element(By.ID, 'dateSubmit').click
        #WebDriverWait(browser, 2)
        #browser.find_element(By.ID, 'dateSubmit').click
        
        # Wait for the data to load
        wait.until_not(EC.presence_of_element_located((By.XPATH, '//div[@class="loading-indicator"]')))
        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'row')))
        wait.until(EC.presence_of_element_located((By.XPATH, '//table[@class="mat-table cdk-table mat-sort ng-star-inserted"]')))
    
    except TimeoutError:
        raise Exception("Timeout Error: Could not load data from URL")
    
    # Extract the data from the table using BeautifulSoup
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    table = soup.select_one('table[class*="mat-table cdk-table mat-sort ng-star-inserted"]')
    rows = table.find_all('tr',{'class':'mat-row cdk-row ng-star-inserted'})
  
    time = []
    temperature = []
    dew_point = []
    humidity = []
    precipitation = []
    wind_speed = []
    wind_dir = []
    condition = []
    
    for row in rows[1:len(rows)-1]:
        cells = row.find_all('td')
       
        time.append(cells[0].text)
        
        temperature_element = cells[1].find('span', class_='wu-value wu-value-to')
        temperature.append(temperature_element.text)
        
        dew_point_element = cells[2].find('span', class_='wu-value wu-value-to')
        dew_point.append(dew_point_element.text)
        
        humidity_element = cells[3].find('span', class_='wu-value wu-value-to')
        humidity.append(humidity_element.text)
        
        wind_dir_element = cells[4].find('span', class_='ng-star-inserted')
        wind_dir.append(wind_dir_element.text)
        
        wind_speed_element = cells[5].find('span', class_='wu-value wu-value-to')
        wind_speed.append(wind_speed_element.text)
        
        precipitation_element = cells[8].find('span', class_='wu-value wu-value-to')
        precipitation.append(precipitation_element.text)
       
        condition_element = cells[9].find('span', class_='ng-star-inserted')
        condition.append(condition_element.text)
        
    #calculating avg
    temperature_avg = fahrenheit_to_celsius(sum(float(t) for t in temperature) / len(temperature))
    precipitation_avg = sum(float(p) for p in precipitation) / len(precipitation)
    wind_avg = mph_to_kmh(sum(float(w) for w in wind_speed) / len(wind_speed))
    humidity_avg = sum(float(r) for r in humidity) / len(humidity)
    dew_point_avg = fahrenheit_to_celsius(sum(float(r) for r in dew_point) / len(dew_point))
    wind_dir_mf = most_frequent_wind_direction(wind_dir)    
    condition_mf = most_frequent_string(condition)
    
   # Close the browser
    #browser.quit()
    data = {'Year': year, 'Month': month, 'Day': day, 
            'Temperature': temperature_avg, 
            'Temp Dew Point': dew_point_avg,
            'Humidity': humidity_avg,
            'Wind Direction': wind_dir_mf, 
            'Wind Speed': wind_avg, 
            'Precipitation': precipitation_avg,
            'Condition': condition_mf}
    return data

In [35]:
def most_frequent_wind_direction(arr):
    wind_directions = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE',
                       'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW',
                       'W', 'WNW', 'NW', 'NNW', 'CALM','VAR']
    
    valid_directions = []
    for direction in arr:
        if direction in wind_directions:
            valid_directions.append(direction)
        else:
            print(f"Warning: Invalid wind direction '{direction}' found in the input array")
    
    count = Counter(valid_directions)
    return count.most_common(1)[0][0]

def most_frequent_string(strings):
    frequency = {}
    max_frequency = 0
    most_frequent_string = None
    for string in strings:
        if string in frequency:
            frequency[string] += 1
        else:
            frequency[string] = 1
        if frequency[string] > max_frequency:
            max_frequency = frequency[string]
            most_frequent_string = string
    return most_frequent_string
 
    # Conversions
def fahrenheit_to_celsius(temp_fahrenheit):
    temp_celsius = (temp_fahrenheit - 32) * 5/9
    return temp_celsius

def mph_to_kmh(mph):
    kmh = mph * 1.609344
    return kmh

In [36]:
#MAIN
browser = webdriver.Firefox()
browser.maximize_window()
#weather_data, problematic_urls = get_weather_data_for_dates7(2010, 8, 4, 2023, 1, 15, browser)
weather_data, problematic_urls = get_weather_data_for_dates7(2010, 9, 20, 2010, 9, 23, browser) #17/09/2010 - problematic
weather_data_df = pd.DataFrame.from_dict(weather_data).transpose()
weather_data_df = weather_data_df.astype({'Year': int, 'Month': int, 'Day': int})

#saving collected data:
weather_data_df.to_csv("weather_data.csv", index=False)
weather_data_df

ERROR:root:An error occurred while getting data for date 2010-09-20 00:00:00: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:392:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:275:16

ERROR:root:Maximum number of retries reached for date 2010-09-20 00:00:00
ERROR:root:An error occurred while getting data for date 2010-09-21 00:00:00: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:392:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:275:16

ERROR:root:Maximum number of retries reached for date 2010-09-21 00:00:00
ERROR:root:An error occurred while getting data fo

ERROR:root:An error occurred while getting data for date 2010-09-23 00:00:00: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:392:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:275:16

ERROR:root:Maximum number of retries reached for date 2010-09-23 00:00:00


KeyError: "Only a column name can be used for the key in a dtype mappings argument. 'Year' not found in columns."

In [ ]:
# Create a DataFrame from the problematic_urls list
problematic_urls_df = pd.DataFrame(problematic_urls, columns=['Date', 'URL'])

# Save the DataFrame to a CSV file
problematic_urls_df.to_csv('problematic_urls_for_weather_data.csv', index=False)
problematic_urls

In [ ]:
   #search_field.click()
        
        #search_field.send_keys("New York, NY")
        #search_field.submit()
       
        # Wait for the auto-complete menu to become visible
        #wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "ui-autocomplete ui-front ui-menu ui-widget ui-widget-content ui-corner-all")))
        #wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".ui-autocomplete.ui-front.ui-menu.ui-widget.ui-widget-content.ui-corner-all")))
        #autocomplete_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".ui-autocomplete")))
        #element = driver.find_elements_by_css_selector('li.needsclick.needsfocus.defcon-.is-city:nth-of-type(2) a span.needsclick.needsfocus.city-name span.ui-autocomplete-term')[0]
        #element.click()
        #try:
            #wait.until(EC.element_to_be_clickable.find_element(By.XPATH, "//span[@class='needsclick needsfocus city-name ng-star-inserted']/span[@class='ui-autocomplete-term'][text()='New York']/following-sibling::text()[1][normalize-space(.)=', NY']").click()
        #except NoSuchElementException:
            #print("No such element was found")

        # Find the auto-complete list
        #autocomplete_list = browser.find_element(By.CSS_SELECTOR, "ul.ui-autocomplete")

        # Find the first option that is not "airports"
        #first_option = autocomplete_list.find_elements(By.CSS_SELECTOR, "li:not(.result-type-airports)")[0]

        # Find the first <li> element with <a> inside it
       # first_result = browser.find_element_by_css_selector("ul.ui-autocomplete li:not(.result-type-airports) a")
        # Click the first result
        #first_result.click()
        #first_option.click()